## Investment Case Group Project

In [312]:
import pandas as pd
import numpy as np
dirpath = 'C:/Users/ankit.bhatia/Documents/GitHub/PythonScripts/data/spark_funds/'
file1 = 'companies.txt'
file2 = 'rounds2.csv'

In [313]:
companies = pd.read_csv(dirpath+file1,sep='\t',encoding='iso-8859-1')
rounds2 = pd.read_csv(dirpath+file2,encoding='iso-8859-1') # Use 'iso-8859-1' for accented characters

In [314]:
companies.head(3)

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/Organization/-Fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/Organization/-Qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
2,/Organization/-The-One-Of-Them-Inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN


In [315]:
rounds2.head(2)

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN


### Table 1.1
#### Q1. How many unique companies are present in rounds2?

In [316]:
# Normal Count
#rounds2.company_permalink.count()

# Unique count which excludes NA as well by default
rounds2.company_permalink.nunique()

90247

#### Q2. How many unique companies are present in companies?

In [317]:
companies.permalink.nunique()

66368

#### Q3. In the companies data frame, which column can be used as the unique key for each company? Write the name of the column.

In [318]:
companies.nunique().sort_values(ascending=False).iloc[0:1]
# nunique() will give unique values for each column, column with maximum unique counts can be considered as unique column#### 

permalink    66368
dtype: int64

#### Q4. Are there any companies in the rounds2 file which are not  present in companies ?

In [319]:
# Check the existacce of a column of dataframe1 in column1 of dataframe2
rounds2[rounds2['company_permalink'].str.lower().isin(companies['permalink'].str.lower())== False]

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
29597,/ORGANIZATION/E-CÃBICA,/funding-round/8491f74869e4fe8ba9c378394f8fbdea,seed,NaN,01-02-2015,NaN
31863,/ORGANIZATION/ENERGYSTONE-GAMES-ÇµÇ³Æ¸¸Æ,/funding-round/b89553f3d2279c5683ae93f45a21cfe0,seed,NaN,09-08-2014,NaN
45176,/organization/huizuche-com-æ ç§ÿè½¦,/funding-round/8f8a32dbeeb0f831a78702f83af78a36,seed,NaN,18-09-2014,NaN
58473,/ORGANIZATION/MAGNET-TECH-Ç£Ç³Ç§Æ,/funding-round/8fc91fbb32bc95e97f151dd0cb4166bf,seed,NaN,16-08-2014,1625585.0
101036,/organization/tipcat-interactive-æ²èÿä¿¡æ¯ç...,/funding-round/41005928a1439cb2d706a43cb661f60f,seed,NaN,06-09-2010,NaN
109969,/ORGANIZATION/WEICHE-TECH-ÅÈ½¦Ç§Æ,/funding-round/f74e457f838b81fa0b29649740f186d8,venture,A,06-09-2015,NaN
113839,/ORGANIZATION/ZENGAME-Ç¦ Æ¸¸Ç§Æ,/funding-round/6ba28fb4f3eadf5a9c6c81bc5dde6cdf,seed,NaN,17-07-2010,NaN


#### Q5. Merge the two data frames so that all  variables (columns)  in the companies frame are added to the rounds2 data frame. Name the merged frame master_frame. How many observations are present in master_frame ?

In [320]:
# lower() the key columns of both dataframes, so they can match in case of diffence in case.
companies['permalink'] = companies['permalink'].str.lower()
rounds2['company_permalink'] = rounds2['company_permalink'].str.lower()
master_frame = pd.merge(left = companies,right = rounds2, how ='inner', left_on='permalink',right_on='company_permalink')
master_frame.shape[0]

114942

### Table 2.1 ( Average Values of Investments for Each of these Funding Types)
#### Q1/2/3/4. Average funding amount of different funding type ?

In [321]:
avg_venture_funding_amount = rounds2[rounds2['funding_round_type']=='venture']['raised_amount_usd'].mean()
print(avg_venture_funding_amount)

avg_angel_funding_amount = rounds2[rounds2['funding_round_type']=='angel']['raised_amount_usd'].mean()
print(avg_angel_funding_amount)

avg_seed_funding_amount = rounds2[rounds2['funding_round_type']=='seed']['raised_amount_usd'].mean()
print(avg_seed_funding_amount)

avg_private_equity_funding_amount = rounds2[rounds2['funding_round_type']=='private_equity']['raised_amount_usd'].mean()
print(avg_private_equity_funding_amount)

11748949.129489528
958694.4697530865
719817.9969071728
73308593.02944215


#### Q5. Considering that Spark Funds wants to invest between 5 to 15 million USD per  investment round, which investment type is the most suitable for them?

In [322]:
# transform is used to calculate  aggregated column with full table
most_suitable = rounds2.groupby(['funding_round_code','funding_round_type'])['raised_amount_usd'].transform(sum)
# Add new column to the rounds2 table.
rounds2['most_suitable'] = most_suitable
most_suitable = rounds2.query('most_suitable >= 5000000 and most_suitable <= 15000000')
most_suitable_data = most_suitable.groupby('funding_round_type')['funding_round_code'].count().sort_values(ascending=False)
most_suitable_data

funding_round_type
seed                9
post_ipo_equity     2
convertible_note    2
debt_financing      1
Name: funding_round_code, dtype: int64

### Table 3.1 ( Analysing the Top 3 English-Speaking Countries)
#### Q1/2/3. Top/Second/Third English speaking country ?

In [334]:
merged9 = pd.merge(left = most_suitable,right = companies, how ='inner', left_on='company_permalink',right_on='permalink')
top9 = merged9.groupby('country_code')['most_suitable'].sum().sort_values(ascending=False)
top9

country_code
USA    76592264.0
TUR    51760000.0
SWE    12940000.0
DNK     5832264.0
Name: most_suitable, dtype: float64

### Table 3.1 ( Sector-wise Investment Analysis)
#### Q1. Total number of Investments (count)

In [364]:
t = companies
t['primary_sector'] = t['category_list'].str.split('|')
type(t['primary_sector'][0])
#t['primary_sector'] = zip(*t['category_list'].map(lambda x: x.split('|')))
#t['primary_sector']
#t


list

In [342]:
t.head()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
2,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN
3,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007
4,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010
